In [ ]:
from aoc.day09 import IntCodeComputer
import numpy as np
import functools

In [ ]:
with open("15-input.txt", "rt") as FILE:
    data = FILE.read()
    data = data.split(",")
    data = [int(d) for d in data]
data[:8]

In [ ]:
functools.reduce(max, [3,5,6])


In [ ]:
class Robot:
    NORTH = (1, (0,1), '^')
    SOUTH = (2, (0,-1), 'v')
    WEST  = (3, (-1,0), '<')
    EAST  = (4, (1,0), '>')
    
    HEADINGS=[NORTH, EAST, SOUTH, WEST]
    
    MOVE_HIT_WALL = 0
    MOVE_SUCCESS = 1
    MOVE_SUCCESS_FOUND_OXYGEN = 2
    
    def print_map(self, moves=False):
        from IPython.display import clear_output
        keys = self.space.keys()
        
        if len(keys) == 0:
            return
        
        x_min = functools.reduce(min, [p[0] for p in keys])
        x_max = functools.reduce(max, [p[0] for p in keys])
        y_min = functools.reduce(min, [p[1] for p in keys])
        y_max = functools.reduce(max, [p[1] for p in keys])
        
#         clear_output()
        for y in range(y_min, y_max+1):
            row = ""
            for x in range(x_min, x_max+1):
                if (x, y) == self.pos:
                    v = self.heading[2]
                elif (x, y) == (0, 0):
                    v = "S"
                elif moves:
                    v = str(self.moves.get((x,y), " "))[-1]
                else:
                    v = self.space.get((x,y), " ")
                row += v
            print(row)
        
    def rotate(self, heading, direction):
        heading = self.HEADINGS.index(heading) + direction
        if heading < 0:
            heading = len(self.HEADINGS)-1
        elif heading >= len(self.HEADINGS):
            heading = 0
        return self.HEADINGS[heading]

    
    def set_pos(self, pos, value):
        self.space[pos] = value
        
    def set_moves(self, pos):
        moves = []
        for h in self.HEADINGS:
            m = self.moves.get(self.calculate_pos(pos, h))
            if m is not None:
                moves.append(m)
        self.moves[pos] = min(moves) + 1
        
        
    def calculate_pos(self, pos, heading):
        pos = np.add(pos, heading[1])
        return (pos[0], pos[1])
    
    
    def receive_output(self, value):
        
        if value == self.MOVE_HIT_WALL:
            wall_pos = self.calculate_pos(self.pos, self.heading) 
            self.set_pos(wall_pos, '#')
            self.heading = self.rotate(self.heading, 1)
            return 0
            
        elif value == self.MOVE_SUCCESS:
            self.pos = self.calculate_pos(self.pos, self.heading) 
            self.set_pos(self.pos, ".")
            self.set_moves(self.pos)
            self.heading = self.rotate(self.heading, -1)
            
            if self.pos == (0,0):
                self.print_map()
                self.print_map(moves=True)
                return -1
            
            return 0
            
        elif value == self.MOVE_SUCCESS_FOUND_OXYGEN:
            self.pos = self.calculate_pos(self.pos, self.heading) 
            self.set_pos(self.pos, "O")
            self.set_moves(self.pos)
            print(self.moves[self.pos])
            return 0
            

    def send_input(self):
        return self.heading[0]
        
    def __init__(self, computer):
        self.pos = (0,0)
        self.heading = self.NORTH
        self.space = {}
        self.moves = {self.pos: 0}
        self.computer = computer
        computer.output_value = self.receive_output
        computer.input_value = self.send_input
        computer.run()

    
    

In [ ]:
computer = IntCodeComputer(list(data), debug=False)
robot = Robot(computer)

# Part 2

Since we already built a complete map using the robot, we can simply calculate positions based on the map in the robot's memory

In [ ]:
def calculate_oxygen_spread(space):
    space = {p: v for p, v in space.items() if v != '#'}
    
    to_check = {p: v for p, v in space.items() if v == '.'}
    while len(to_check) > 0:
        for pos in to_check.keys():
            moves = []
            for h in Robot.HEADINGS:
                m = space.get(robot.calculate_pos(pos, h))
                if m is None:
                    pass
                elif m == 'O':
                    moves.append(0)
                elif isinstance(m, int):
                    moves.append(m)
            if len(moves)>0:
                space[pos] = min(moves) + 1
        to_check = {p: v for p, v in space.items() if v == '.'}
        
        
    values = [v for v in space.values() if isinstance(v, int)]
    return max(values)


calculate_oxygen_spread(robot.space)